# Data Preprocessing & Feature Engineering – Task Sheet

**Author:** Ahmed Hassan
  
**Date:** 2025-08-29

**Instructions**
- Each task appears in this single notebook.
- For every task: **Introduction → Data loading & preprocessing steps only → Results & observations**.
- Keep things concise. Only minimal visuals where explicitly required (e.g., outlier visualization).
- Some datasets (Kaggle) require manual download. The code tries to download from public sources when possible; otherwise, put files under `./data/` as instructed below.

---

### ⚙️ Setup Cell (Run once)


In [ ]:
# Install (if needed) and import libraries
# If running in an environment where you cannot install packages, comment-out pip lines.
# !pip install -q pandas numpy scikit-learn imbalanced-learn scipy matplotlib nltk requests

import os
import io
import zipfile
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder, PowerTransformer
)
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import RFE, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

from scipy import stats

import matplotlib.pyplot as plt

# NLTK for text preprocessing (Task 12)
import nltk
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt")
try:
    nltk.data.find("corpora/stopwords")
except LookupError:
    nltk.download("stopwords")
try:
    nltk.data.find("wordnet")
except LookupError:
    nltk.download("wordnet")

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

def try_read_csv(local_name: str, url: str = None, **read_kwargs) -> pd.DataFrame:
    # Try loading a CSV from ./data/local_name; if not present and url is provided,
    # try to download via pandas.read_csv(url). If both fail, raise a clear error.
    local_path = os.path.join(DATA_DIR, local_name)
    if os.path.exists(local_path):
        return pd.read_csv(local_path, **read_kwargs)
    if url:
        try:
            return pd.read_csv(url, **read_kwargs)
        except Exception as e:
            raise FileNotFoundError(
                f"Could not download from URL. Place the file at {local_path}. Original error: {e}"
            )
    raise FileNotFoundError(
        f"Place the required file at {local_path} or supply a valid URL."
    )

def report_missing(df: pd.DataFrame, title: str = "Missing Values Report"):
    miss = df.isna().sum().sort_values(ascending=False)
    print(f"\n=== {title} ===")
    print(miss[miss > 0])
    if miss.sum() == 0:
        print("No missing values detected.")

print('Setup complete.')


## Task 1: Handling Missing Data – Titanic Dataset

**Introduction (overview + chosen techniques):**  
We identify and treat missing values using mean/median (numeric), mode (categorical), and selective dropping when a column is mostly missing.

**Dataset:** Titanic (Kaggle). Fallback: Seaborn's `titanic` sample if available.  
**What to show:** Missing counts before/after; dataset shape before/after.


In [ ]:
# --- Task 1 Code ---
# Preferred: Kaggle 'train.csv' under ./data/train.csv (rename to titanic_train.csv)
# Fallback: try seaborn built-in if internet available (not guaranteed).

import pandas as pd

df_titanic = None
local_path = os.path.join(DATA_DIR, "titanic_train.csv")  # rename your Kaggle train.csv to titanic_train.csv

if os.path.exists(local_path):
    df_titanic = pd.read_csv(local_path)
else:
    try:
        import seaborn as sns
        df_titanic = sns.load_dataset("titanic")  # schema differs slightly
    except Exception:
        raise FileNotFoundError(
            "Provide Titanic train.csv as ./data/titanic_train.csv or ensure seaborn can load the sample dataset."
        )

print("Shape before:", df_titanic.shape)
report_missing(df_titanic, "Titanic Missing Before")

# Separate numeric and categorical
num_cols = df_titanic.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df_titanic.select_dtypes(exclude=[np.number]).columns.tolist()

# Impute numeric: median
num_imputer = SimpleImputer(strategy="median")
df_titanic[num_cols] = num_imputer.fit_transform(df_titanic[num_cols])

# Impute categorical: most_frequent
if cat_cols:
    cat_imputer = SimpleImputer(strategy="most_frequent")
    df_titanic[cat_cols] = cat_imputer.fit_transform(df_titanic[cat_cols])

report_missing(df_titanic, "Titanic Missing After")
print("Shape after:", df_titanic.shape)


## Task 2: Encoding Categorical Variables – Car Evaluation Dataset

**Introduction:** Categorical variables must be encoded to numeric form. We compare **Label Encoding** (ordinal codes) vs **One-Hot Encoding** (binary indicators).

**Dataset:** UCI Car Evaluation.  
**What to show:** Shapes and sample heads for both encodings.


In [ ]:
# --- Task 2 Code ---
uci_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
cols = ["buying","maint","doors","persons","lug_boot","safety","class"]

car_df = try_read_csv("car.data", uci_url, header=None, names=cols)

# Label Encoding (including target for demonstration)
le = LabelEncoder()
car_le = car_df.apply(le.fit_transform)

print("Label Encoded shape:", car_le.shape)
print(car_le.head())

# One-hot Encoding (exclude target when appropriate; here we encode all features, keep target separate)
X = car_df.drop(columns=["class"])
y = car_df["class"]

ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = pd.DataFrame(ohe.fit_transform(X), columns=ohe.get_feature_names_out(X.columns))

print("One-hot Encoded shape:", X_ohe.shape)
print(X_ohe.head())


## Task 3: Feature Scaling – Wine Quality Dataset

**Introduction:** Compare **Normalization (MinMaxScaler)** and **Standardization (StandardScaler)** on numeric features.

**Dataset:** UCI Wine Quality (red wine).  
**What to show:** Summary statistics before/after; basic distribution change (no heavy plots).


In [ ]:
# --- Task 3 Code ---
wine_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine = try_read_csv("winequality-red.csv", wine_url, sep=";")

features = wine.drop(columns=["quality"])
minmax = MinMaxScaler()
std = StandardScaler()

features_minmax = pd.DataFrame(minmax.fit_transform(features), columns=features.columns)
features_std = pd.DataFrame(std.fit_transform(features), columns=features.columns)

print("Original describe:\n", features.describe().T[["mean","std","min","max"]])
print("\nMinMax describe:\n", features_minmax.describe().T[["mean","std","min","max"]])
print("\nStandardized describe:\n", features_std.describe().T[["mean","std","min","max"]])


## Task 4: Handling Outliers – Boston Housing Dataset

**Introduction:** Detect outliers via **Z-score** and **IQR**. Provide simple visual (boxplot) as required.

**Dataset:** Boston Housing (UCI).  
**What to show:** Counts removed via each method; shape before/after; one boxplot for a representative column.


In [ ]:
# --- Task 4 Code ---
boston_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
boston_cols = ["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"]

boston = try_read_csv("housing.data", boston_url, delim_whitespace=True, header=None, names=boston_cols)

print("Shape before:", boston.shape)

# Simple boxplot for 'RM'
plt.figure()
boston["RM"].plot(kind="box", title="Boxplot of RM (rooms)")
plt.show()

# Z-score method (remove rows where any feature has |z| > 3)
z = np.abs(stats.zscore(boston.select_dtypes(include=[np.number])))
z_mask = (z < 3).all(axis=1)
boston_z = boston[z_mask]
print("After Z-score filter (|z|<3):", boston_z.shape)

# IQR method for LSTAT (example feature)
Q1 = boston["LSTAT"].quantile(0.25)
Q3 = boston["LSTAT"].quantile(0.75)
IQR = Q3 - Q1
mask_iqr = (boston["LSTAT"] >= Q1 - 1.5*IQR) & (boston["LSTAT"] <= Q3 + 1.5*IQR)
boston_iqr = boston[mask_iqr]
print("After IQR filter on LSTAT:", boston_iqr.shape)


## Task 5: Advanced Data Imputation – Retail Sales Dataset

**Introduction:** Use **KNNImputer** and **MICE (IterativeImputer)** for smarter imputations that consider feature relationships.

**Dataset:** Retail Sales (Kaggle – provide your CSV as `retail_sales.csv`).  
**What to show:** Missing values before/after for each technique.


In [ ]:
# --- Task 5 Code ---
retail_path = os.path.join(DATA_DIR, "retail_sales.csv")
if os.path.exists(retail_path):
    retail = pd.read_csv(retail_path)
else:
    np.random.seed(42)
    dates = pd.date_range("2021-01-01", periods=200, freq="D")
    retail = pd.DataFrame({
        "date": np.random.choice(dates, size=500),
        "store": np.random.choice(["A","B","C"], size=500),
        "item": np.random.choice(["SKU1","SKU2","SKU3","SKU4"], size=500),
        "price": np.random.uniform(5, 50, size=500),
        "promo": np.random.choice([0,1], size=500, p=[0.7, 0.3]),
        "sales": np.random.poisson(20, size=500).astype(float)
    })
    # Inject missingness
    for col in ["price","promo","sales"]:
        idx = np.random.choice(retail.index, size=60, replace=False)
        retail.loc[idx, col] = np.nan

print("Missing BEFORE:")
report_missing(retail)

# Encode categorical to numeric for imputation
retail_enc = retail.copy()
for c in retail_enc.select_dtypes(exclude=[np.number]).columns:
    retail_enc[c] = LabelEncoder().fit_transform(retail_enc[c].astype(str))

num_cols = retail_enc.select_dtypes(include=[np.number]).columns

# KNN Imputer
knn_imp = KNNImputer(n_neighbors=5)
retail_knn = retail_enc.copy()
retail_knn[num_cols] = knn_imp.fit_transform(retail_knn[num_cols])

print("\nMissing AFTER KNN:")
report_missing(retail_knn)

# MICE / Iterative Imputer
mice_imp = IterativeImputer(random_state=42, sample_posterior=False, max_iter=10)
retail_mice = retail_enc.copy()
retail_mice[num_cols] = mice_imp.fit_transform(retail_mice[num_cols])

print("\nMissing AFTER MICE:")
report_missing(retail_mice)


## Task 6: Feature Engineering – Heart Disease Dataset

**Introduction:** Create useful derived features such as age groups, cholesterol categories, and composite risk flags.

**Dataset:** UCI Heart Disease.  
**What to show:** `head()` with new features.


In [ ]:
# --- Task 6 Code ---
uci_heart_url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/heart/heart.csv"
heart = try_read_csv("heart.csv", uci_heart_url)

# Feature Engineering
heart["age_group"] = pd.cut(heart["age"], bins=[0,40,55,70,120],
                            labels=["<=40","41-55","56-70",">70"], right=True, include_lowest=True)

heart["chol_cat"] = pd.cut(heart["chol"], bins=[0,200,240,600],
                           labels=["desirable","borderline_high","high"], include_lowest=True)

# Simplified risk score example
heart["risk_bp_chol"] = ((heart["trestbps"] > 130).astype(int) + (heart["chol"] > 240).astype(int))

# Composite risk flag example
heart["risk_flag"] = (
    ((heart["age"] > 55).astype(int) +
     (heart["trestbps"] > 140).astype(int) +
     (heart["chol"] > 240).astype(int) +
     (heart["exang"] == 1).astype(int)) >= 2
).astype(int)

print(heart[["age","trestbps","chol","age_group","chol_cat","risk_bp_chol","risk_flag"]].head())


## Task 7: Variable Transformation – Bike Sharing Dataset

**Introduction:** Reduce skewness using **log**, **square-root**, and **Box-Cox** transforms.

**Dataset:** UCI Bike Sharing (day.csv).  
**What to show:** Skewness before/after for key skewed columns (e.g., `cnt`).


In [ ]:
# --- Task 7 Code ---
bike_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"

# Try to download and read 'day.csv' from the zip; else expect ./data/day.csv
day_path = os.path.join(DATA_DIR, "day.csv")
if not os.path.exists(day_path):
    try:
        import requests
        from io import BytesIO
        resp = requests.get(bike_url, timeout=30)
        zf = zipfile.ZipFile(BytesIO(resp.content))
        with zf.open("day.csv") as f:
            day = pd.read_csv(f)
    except Exception:
        day = try_read_csv("day.csv")
else:
    day = pd.read_csv(day_path)

def skew_report(s, name):
    print(f"{name} skew: {s.skew():.4f}")

skew_report(day["cnt"], "Original cnt")

# Log (use log1p to handle zeros)
day["cnt_log"] = np.log1p(day["cnt"])
skew_report(day["cnt_log"], "Log cnt")

# Square-root
day["cnt_sqrt"] = np.sqrt(day["cnt"])
skew_report(day["cnt_sqrt"], "Sqrt cnt")

# Box-Cox (requires strictly positive; cnt >= 0, add +1)
pt = PowerTransformer(method="box-cox", standardize=False)
day["cnt_boxcox"] = pt.fit_transform((day["cnt"] + 1).values.reshape(-1,1)).ravel()
skew_report(day["cnt_boxcox"], "Box-Cox cnt")


## Task 8: Feature Selection – Diabetes Dataset

**Introduction:** Select important features via **Correlation**, **Mutual Information**, and **RFE**.

**Dataset:** Pima Indians Diabetes (provide `diabetes.csv` locally or use mirror).  
**What to show:** Selected features from each method.


In [ ]:
# --- Task 8 Code ---
pima_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
pima_cols = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age","Outcome"]

diabetes = try_read_csv("diabetes.csv", pima_url, header=None, names=pima_cols)

X = diabetes.drop(columns=["Outcome"])
y = diabetes["Outcome"]

# 1) Correlation (absolute)
corr = X.join(y).corr(numeric_only=True)["Outcome"].drop("Outcome").abs().sort_values(ascending=False)
print("Correlation with Outcome:\n", corr)

# 2) Mutual Information
mi = mutual_info_classif(X, y, random_state=42)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print("\nMutual Information:\n", mi_series)

# 3) RFE with Logistic Regression
lr = LogisticRegression(max_iter=1000, solver="liblinear")
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(X, y)
selected_rfe = X.columns[rfe.support_]
print("\nRFE Selected (5):", list(selected_rfe))


## Task 9: Handling Imbalanced Data – Credit Card Fraud Detection

**Introduction:** Demonstrate resampling using **SMOTE**, **ADASYN**, and **undersampling**.

**Dataset:** Kaggle Credit Card Fraud (`creditcard.csv`).  
**What to show:** Class distribution before/after each technique.


In [ ]:
# --- Task 9 Code ---
path_cc = os.path.join(DATA_DIR, "creditcard.csv")
if os.path.exists(path_cc):
    cc = pd.read_csv(path_cc)
    X = cc.drop(columns=["Class"])
    y = cc["Class"]
else:
    from sklearn.datasets import make_classification
    X, y = make_classification(n_samples=5000, n_features=20, n_informative=3, n_redundant=2,
                               weights=[0.98, 0.02], random_state=42)
    X = pd.DataFrame(X)
    y = pd.Series(y, name="Class")

print("Class distribution BEFORE:\n", y.value_counts(normalize=True))

# SMOTE
sm = SMOTE(random_state=42)
X_sm, y_sm = sm.fit_resample(X, y)
print("\nAFTER SMOTE:\n", y_sm.value_counts(normalize=True))

# ADASYN
ad = ADASYN(random_state=42)
X_ad, y_ad = ad.fit_resample(X, y)
print("\nAFTER ADASYN:\n", y_ad.value_counts(normalize=True))

# Undersampling
rus = RandomUnderSampler(random_state=42)
X_ru, y_ru = rus.fit_resample(X, y)
print("\nAFTER Random Undersampling:\n", y_ru.value_counts(normalize=True))


## Task 10: Combining Multiple Datasets – MovieLens Dataset

**Introduction:** Merge ratings, movies, and (if available) users/links.

**Dataset:** MovieLens (e.g., **ml-latest-small**).  
**What to show:** Merged dataset shape and `head()`.


In [ ]:
# --- Task 10 Code ---
ml_dir = os.path.join(DATA_DIR, "ml-latest-small")
ratings_path = os.path.join(ml_dir, "ratings.csv")
movies_path = os.path.join(ml_dir, "movies.csv")
links_path = os.path.join(ml_dir, "links.csv")

if not (os.path.exists(ratings_path) and os.path.exists(movies_path)):
    # Try download
    try:
        import requests
        url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
        r = requests.get(url, timeout=30)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(DATA_DIR)
    except Exception:
        print("Please place MovieLens 'ml-latest-small' in ./data/")
        
ratings = pd.read_csv(ratings_path) if os.path.exists(ratings_path) else None
movies = pd.read_csv(movies_path) if os.path.exists(movies_path) else None
links  = pd.read_csv(links_path) if os.path.exists(links_path) else None

if ratings is None or movies is None:
    raise FileNotFoundError("ratings.csv/movies.csv not found. Ensure MovieLens is available in ./data/.")

merged = ratings.merge(movies, on="movieId", how="left")
if links is not None:
    merged = merged.merge(links, on=["movieId"], how="left")

print("Merged shape:", merged.shape)
print(merged.head())


## Task 11: Dimensionality Reduction – MNIST Dataset

**Introduction:** Reduce dimensionality with **PCA** (retain 95% variance) and visualize a basic **t-SNE** 2D embedding (no labels/insights beyond shape print).

**Dataset:** MNIST (via `keras.datasets.mnist`).


In [ ]:
# --- Task 11 Code ---
from tensorflow.keras.datasets import mnist

# Load
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Flatten
X_flat = X.reshape((X.shape[0], -1)).astype("float32") / 255.0
print("MNIST flattened shape:", X_flat.shape)

# PCA retain 95% variance
pca = PCA(n_components=0.95, svd_solver="full", random_state=42)
X_pca = pca.fit_transform(X_flat)
print("PCA shape (95% var):", X_pca.shape)

# t-SNE (subset for speed)
idx = np.random.choice(len(X_pca), size=5000, replace=False)
tsne = TSNE(n_components=2, init="random", random_state=42, perplexity=30, learning_rate="auto")
X_tsne = tsne.fit_transform(X_pca[idx])

print("t-SNE 2D embedding shape:", X_tsne.shape)


## Task 12: Text Preprocessing – IMDB Movie Reviews Dataset

**Introduction:** Clean and preprocess raw text: lowercasing, stopword removal, tokenization, stemming/lemmatization.

**Dataset:** Provide a CSV `IMDB_Dataset.csv` with columns `review` and `sentiment` (Kaggle format).  
**What to show:** Example before/after for 3 reviews.


In [ ]:
# --- Task 12 Code ---
path_imdb = os.path.join(DATA_DIR, "IMDB_Dataset.csv")
if not os.path.exists(path_imdb):
    raise FileNotFoundError("Place IMDB_Dataset.csv in ./data/ with columns: review, sentiment")

imdb = pd.read_csv(path_imdb)

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemm = WordNetLemmatizer()

def preprocess_text(s: str, use_stem=False, use_lemma=True):
    s = str(s).lower()
    tokens = word_tokenize(s)
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]
    if use_stem:
        tokens = [stemmer.stem(t) for t in tokens]
    if use_lemma:
        tokens = [lemm.lemmatize(t) for t in tokens]
    return " ".join(tokens)

# Show 3 examples before/after
sample = imdb.sample(3, random_state=42).copy()
sample["cleaned"] = sample["review"].apply(preprocess_text)
print(sample[["review","cleaned","sentiment"]])


## Task 13: Time-Series Preprocessing – Air Quality Dataset

**Introduction:** Handle missing timestamps, resample, and smooth.

**Dataset:** UCI Air Quality (place `AirQualityUCI.csv` or similar in `./data/`).  
**What to show:** Head of the processed series.


In [ ]:
# --- Task 13 Code ---
aq_local = os.path.join(DATA_DIR, "AirQualityUCI.csv")
if os.path.exists(aq_local):
    aq = pd.read_csv(aq_local, sep=";", decimal=",")
    aq = aq.dropna(axis=1, how="all")
    if "Date" in aq.columns and "Time" in aq.columns:
        dt = pd.to_datetime(aq["Date"] + " " + aq["Time"], errors="coerce", dayfirst=True)
        aq = aq.assign(datetime=dt).drop(columns=["Date","Time"])
    elif "datetime" in aq.columns:
        aq["datetime"] = pd.to_datetime(aq["datetime"], errors="coerce")
    else:
        raise ValueError("Provide Date/Time or datetime columns.")
else:
    # Synthesize hourly data with gaps for demonstration
    rng = pd.date_range("2022-01-01", periods=500, freq="H")
    aq = pd.DataFrame({"datetime": rng, "CO(GT)": np.random.normal(1.2, 0.3, size=len(rng))})
    drop_idx = np.random.choice(aq.index, size=50, replace=False)
    aq = aq.drop(index=drop_idx).sort_values("datetime")

# Set index and reindex to continuous hourly frequency
aq = aq.set_index("datetime").sort_index()
full_index = pd.date_range(aq.index.min(), aq.index.max(), freq="H")
aq_reindexed = aq.reindex(full_index)

# Impute missing via forward-fill then mean fill
aq_filled = aq_reindexed.ffill().fillna(aq_reindexed.mean(numeric_only=True))

# Resample daily mean
aq_daily = aq_filled.resample("D").mean(numeric_only=True)

# Smooth with 7-day rolling mean
aq_smoothed = aq_daily.rolling(window=7, min_periods=1).mean()

print("Original head:\n", aq.head())
print("\nDaily head:\n", aq_daily.head())
print("\nSmoothed head:\n", aq_smoothed.head())


---

### ✅ Notes
- Where internet downloads are not possible, place the required files in `./data/` as indicated in each task.
- Keep the notebook outputs minimal: only the required prints/one simple plot for outliers.
- After running all cells, push this `.ipynb` to your GitHub repo.

Good luck!
